In [1]:
using Revise, Pkg
Pkg.activate(".")

In [22]:
using Napari, ThreeDeconv, FFTW, Images, Printf, Statistics, HDF5, CUDA

In [9]:
img = Float32.(load("data.tif"));
psf = Float32.(load("psf.tif"));
img ./= maximum(img);

In [27]:
γ, σ = ThreeDeconv.noise_estimation(img, maxnum_pairs = 200)
@printf "Gain: %.1f, Read noise std.: %.1f \n" γ σ


options =
    ThreeDeconv.DeconvolutionOptions(max_iters = 300, show_trace = true, check_every = 50)
reg = 1e-7

Computing local noise variance.
Initializing parameters by least-squares.
Initialization done.
Starting likelihood maximization.
Finished the maximization.
Estimated noise parameters are a = 0.001, b = 0.000.
Estimated parameters for Poisson-Gaussian noise model
γ: 0.0011922871862607468, σ: 0.008916181746551226
Gain: 0.0, Read noise std.: 0.0 


1.0e-7

In [11]:
using ThreeDeconv

In [29]:
ThreeDeconv.to_gpu_or_not_to_gpu(randn((2,2)))

2×2 CuArray{Float64, 2}:
  1.17254  -0.871916
 -2.27469  -0.319056

In [ ]:
@time result = ThreeDeconv.deconvolve(img, psf, γ, σ, reg, options = options)

No clipping performed.


In [18]:
h5write("/home/fxw/Nextcloud/Uni/DeconvOptim.jl/good_data/ThreeDeconv.h5", "res", result.x)